In [9]:
# Import all required modules

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
# Create RDD (Resilient Distributed Dataset)

MsBrand = spark.read.option("inferSchema", "true").csv("MsBrand.csv", header=True)
MsRamen = spark.read.option("inferSchema", "true").csv("MsRamen.csv", header=True)
MsCountry = spark.read.option("inferSchema", "true").csv("MsCountry.csv", header=True)
MsRamenStyle = spark.read.option("inferSchema", "true").csv("MsRamenStyle.csv", header=True)

In [ ]:
# Halo mau di show RDD (SELECT *)

MsBrand.show()
MsRamen.show()
MsCountry.show()
MsRamenStyle.show()

# Parameter 1 = top n row
# Parameter 2 = truncate

MsBrand.show(10, True)

In [ ]:
# Select specific column with filtering condition (Query Builder)

MsRamen.select("RamenId", "RamenName", "RamenPrice").join().filter().show()

#.show() parameter can be called quantity
MsRamen.select("RamenId", "RamenName", "RamenPrice").show(5)

# .join() JOIN TABLE
MsRamen.join(MsRamenStyle, MsRamenStyle.RamenStyleId == MsRamen.RamenStyleId).select("RamenId", "RamenName", "RamenPrice", "RamenStyleName").filter(MsRamenStyle.RamenStyleName == 'Cup').show()

# .filter() WHERE()
MsRamen
.select("RamenId", "RamenName", "RamenPrice")
.filter(MsRamen["RamenName"].startswith("Noodles"))
.show()

# INFO quiz is not necessary have to used the SQL Query Builder

# You can use basic QUERY -> SELECT TOP 5 RamenId, RamenName, RamenPrice FROM MsRamen

In [ ]:
# RAW QUERY -> Masuk QUIZ
# Temporary Table for easier manipulating SQL like syntax

MsRamen.createOrReplaceTempView("MsRamen")
MsBrand.createOrReplaceTempView("MsBrand")
MsCountry.createOrReplaceTempView("MsCountry")
MsRamenStyle.createOrReplaceTempView("MsRamenStyle")

# Query:
# 1. SELECT all data from MsRamen

spark.sql("""
    SELECT * FROM MsRamen LIMIT 20  
""").show(20)

# 2. Display RamenStyleName and average of the ramen price for every even RamenId, Order it by average price DESC and only show 10 first data

# Aggregate function = SUM/TOTAL (SUM) , Average (AVG) , Amount/COUNT, MAX. MIN => GROUP BY
spark.sql(""" 
    SELECT RamenStyleName,
    ROUND(AVG(RamenPrice), 2) AS 'AveragePrice'
    FROM MsRamen AS mr
    JOIN MsRamenStyle mrs ON mr.RamenStyleId = mrs.RamenStyleId
    WHERE mr.RamenId % 2 = 0
    GROUP BY RamenStyleName
    ORDER BY AveragePrice DESC
    LIMIT 10
""").show(10)

# ROUND = Pembulatan angka dibelakang koma
# WHERE locate above GROUP BY and below FROM / JOIN
# HAVING is locate below GROUP BY

# 3. Get All data of Ramen. IT includes RamenName, RamenPrice, and RamenCategory. RamenCategory is determined by comparing their price:
# 0 - 14999 => 'small'
# 15000 - 24999 => 'regular'
# 25000+ => 'premium', from condition above we need to use CASE WHEN

spark.sql("""
    SELECT RamenName,
    RamenPrice,
    CASE
        -- WHEN [kondisi] THEN [expression/value]
        WHEN RamenPrice >= 25000 THEN 'premium'
        WHEN RamenPrice >= 15000 THEN 'regular'
        ELSE 'small
    END AS 'RamenCategory'
    FROM MsRamen
""").show(50)